In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import geopandas as gpd
import folium
from shapely.geometry import Point

ImportError: DLL load failed: The specified module could not be found.

In [2]:
diagnostics = pd.read_csv('./data/VehicleDiagnosticOnboardData.csv', header = None)
codes = pd.read_excel('./data/Service_Fault_Codes_1_0_0_167.xlsx')
faults = pd.read_csv('./data/J1939Faults.csv', header = None)

C:\Users\jnske\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
columns1 = ['RecordID', 'Name', 'Value', 'FaultID']
diagnostics.columns = columns1
# diagnostics = diagnostics.drop('Index', axis = 1)

In [4]:
diagnostics.head()

,RecordID,Name,Value,FaultID
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [5]:
codes.head()

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...


In [6]:
columns2 = ['FaultID', 'ESS_ID', 'eventTimeStamp', 'eventDescription', 'actionDescription', 'ecuSoftwareVersion', 
            'ecuSerialNumber', 'ecuModel', 'ecuMake', 'ecuSource', 'SPN', 'FMI', 'Active', 'activeTransitionCount', 
            'faultValue', 'EquipmentID', 'MCTNumber', 'Latitude', 'Longitude', 'locationTimeStamp']
faults.columns = columns2
faults.head()

,FaultID,ESS_ID,eventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,SPN,FMI,Active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,locationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [7]:
faults.ecuMake.value_counts()

CMMNS              409642
unknown            294542
PACCR              225482
BNDWS               63873
PCAR                18964
VOLVO                7252
EATON                6474
?????                 686
????S                 526
????R                 327
?MMNS                 270
?CAR                  151
?ACCR                  39
?NDWS                  15
?????MX                 6
?????MX16U13D13         4
??MNS                   3
???R                    1
Name: ecuMake, dtype: int64

In [8]:
faults.loc[faults['ecuMake'].isin(['PCAR', '????R',
                                 '?CAR', '?ACCR', '???R']), 'ecuMake']="PACCR"
faults.loc[faults['ecuMake'].isin(['?MMNS', '??MNS']), 'ecuMake']="CMMNS"
faults.loc[faults['ecuMake']=='?NDWS', 'ecuMake']="BNDWS"

In [9]:
faults['EquipmentID'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
faults['EquipmentID']=faults['EquipmentID'].astype(float)
faults['EquipmentID']=faults['EquipmentID'].astype(int)

In [10]:
faults=faults.loc[(faults["ecuMake"]=="CMMNS"),:] 

In [11]:
joined_df = faults.set_index('FaultID').join(diagnostics.pivot(
    index = 'FaultID', 
    columns = 'Name', 
    values = 'Value')).reset_index().set_index('FaultID')

In [12]:
codes['Lamp Device'].value_counts()

Warning            4972
Stop / Shutdown     880
None                871
Maintenance         401
Name: Lamp Device, dtype: int64

In [13]:
codes_subset = codes[['SPN', 'J1939 FMI', 'Lamp Device', 'Cummins Description']]

In [14]:
codes_subset.columns = ['SPN', 'FMI', 'Lamp Device', 'Cummins Description']

In [15]:
codes_subset['Cummins Description'].isna().sum()

0

In [16]:
codes_subset['combo_code'] = 0
codes_subset['combo_code'] = codes_subset['SPN'].map(str) + codes_subset['FMI'].map(str)
codes_subset.head()

C:\Users\jnske\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\jnske\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SPN,FMI,Lamp Device,Cummins Description,combo_code
0,629,12,Stop / Shutdown,Engine Control Module Critical Internal Failur...,62912
1,635,7,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,6357
2,635,3,Warning,Engine Timing Actuator Driver Circuit - Voltag...,6353
3,635,4,Warning,Engine Timing Actuator Driver Circuit - Voltag...,6354
4,612,2,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,6122


In [20]:
maintenance = joined_df.loc[((joined_df['Latitude'] == 36.0666667) & (joined_df['Longitude'] == -86.4347222)) | 
                                ((joined_df['Latitude'] == 35.5883333) & (joined_df['Longitude'] == -86.4438888)) | 
                                ((joined_df['Latitude'] == 36.1950) & (joined_df['Longitude'] == -83.174722))]
maintenance.shape

(19, 43)

In [21]:
joined_df.head()

,ESS_ID,eventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,SPN,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultID,,,,,,,,,,,,,,,,,,,,,
6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,...,NaN,True,111.2,1023,NaN,NaN,13.6022,3276.75,NaN,6.67
8,990441,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,111,...,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
9,990442,2015-02-21 11:40:22.000,High (Severity Low) Water In Fuel Indicator,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,97,...,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
10,990446,2015-02-21 11:41:55.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
11,990459,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,...,NaN,True,78.8,1023,NaN,NaN,64.82649,3276.75,NaN,17.69


In [22]:
joined_df['EquipmentID'].value_counts()

1814         10260
1816          8018
1601          7064
1592          6831
1815          6355
1803          6336
1809          6098
1600          5668
1603          5579
1589          5481
1559          5326
1808          5287
1804          5222
1562          5209
1818          5041
1820          4880
1806          4798
1554          4375
1573          4202
1594          4122
1563          3857
1812          3812
1569          3804
1556          3785
1564          3178
1811          3163
1805          3095
1819          3084
1590          3057
1577          3043
             ...  
105399896        2
2049             2
2069             2
2198             2
1430             2
1426             2
1354             2
2224             1
105363528        1
105355995        1
105311799        1
105349656        1
105411467        1
1428             1
2197             1
105337895        1
1672             1
1548             1
105411463        1
105322231        1
105349500        1
105340140   

In [23]:
# codes.columns.values

In [24]:
# codes_subset = codes[['SPN', 'J1939 FMI', 'Lamp Device', 'Cummins Description']]

In [25]:
# codes_subset.head()

In [26]:
# codes_subset.columns = ['SPN', 'FMI', 'Lamp Device', 'Cummins Description']

In [27]:
# codes_subset['Cummins Description'].isna().sum()

In [28]:
# codes_subset['combo_code'] = 0
# codes_subset['combo_code'] = codes_subset['SPN'].map(str) + codes_subset['FMI'].map(str)
# codes_subset.head()

In [29]:
# codes_subset['combo_code'].value_counts()

In [30]:
# codes_subset.loc[codes_subset['combo_code']=='52428631']

In [31]:
# joined_df['combo_code'] = 0
# joined_df['combo_code'] = joined_df['SPN'].map(str) + joined_df['FMI'].map(str)
# joined_df.head()

In [32]:
# joined_df.shape

In [33]:
# joined_cummins = pd.merge(joined_df, codes_subset, on = ['combo_code'], how = 'inner')

In [34]:
# joined_cummins.head()

In [35]:
# joined_cummins.shape

In [36]:
derated = joined_df.loc[((joined_df['SPN'] == 5246) & (joined_df.FMI == 0)) | ((joined_df['SPN'] == 5246) & (joined_df['FMI'] == 15)) | 
              ((joined_df['SPN'] == 5246) & (joined_df['FMI'] == 16)) | ((joined_df['SPN'] == 1569)),:]
derated.head()

,ESS_ID,eventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,SPN,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultID,,,,,,,,,,,,,,,,,,,,,
41,990856,2015-02-21 12:06:22.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,...,32,True,51.8,18431,False,NaN,51.97187,3276.75,0,0.58
46,990931,2015-02-21 12:10:51.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,...,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN
250,993542,2015-02-21 15:01:49.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,...,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN
291,994045,2015-02-21 15:35:45.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,...,32,True,91.4,18431,False,NaN,40.73865,3276.75,0,15.66
307,994237,2015-02-21 15:51:58.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,...,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
derated.shape

(9223, 43)

In [38]:
joined_df['derated'] = 0

In [39]:
joined_df.loc[((joined_df['SPN'] == 5246) & (joined_df.FMI == 0)) | ((joined_df['SPN'] == 5246) & (joined_df['FMI'] == 15)) | 
              ((joined_df['SPN'] == 5246) & (joined_df['FMI'] == 16)) | ((joined_df['SPN'] == 1569)), 'derated'] = 1

In [40]:
joined_df.shape

(409915, 44)

In [41]:
joined_df['combo_code'] = 0
joined_df['combo_code'] = joined_df['SPN'].map(str) + '/' + joined_df['FMI'].map(str)
joined_df.head()

,ESS_ID,eventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,SPN,...,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,derated,combo_code
FaultID,,,,,,,,,,,,,,,,,,,,,
6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,...,111.2,1023,NaN,NaN,13.6022,3276.75,NaN,6.67,0,111/17
8,990441,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,111,...,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,0,111/17
9,990442,2015-02-21 11:40:22.000,High (Severity Low) Water In Fuel Indicator,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,97,...,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,0,97/15
10,990446,2015-02-21 11:41:55.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,...,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,0,111/17
11,990459,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,...,78.8,1023,NaN,NaN,64.82649,3276.75,NaN,17.69,0,111/17


In [42]:
not_maintenance = joined_df.loc[~(joined_df['combo_code'].isin(maintenance))]

In [43]:
not_maintenance.combo_code.value_counts()

111/17      283294
111/18       24135
97/15        13436
1569/31       8694
97/3          5514
1761/17       3621
111/3         3374
3464/7        2669
641/9         2216
3226/20       1984
4364/18       1590
3216/20       1401
630/12        1283
2623/4        1179
91/4          1169
3610/2        1136
3251/16       1133
3216/4        1086
157/18        1029
96/3          1027
5742/9         988
5848/4         985
3251/2         983
5743/9         948
4339/7         943
111/1          905
3362/31        891
96/9           888
641/11         873
411/2          831
             ...  
4331/18          2
255/2            2
32000/0          2
3327/16          2
6713/13          2
3521/18          2
3490/4           2
65535/3          2
3583/4           2
5743/11          2
38/9             2
5491/3           2
1322/11          1
4752/7           1
3556/1           1
65535/17         1
2629/2           1
190/11           1
3241/2           1
4094/18          1
4813/3           1
5745/2      

In [53]:
# removing codes for low coolant
final_df = not_maintenance.loc[((not_maintenance['combo_code'] != '111/17') & (not_maintenance['combo_code'] != '111/18'))]
final_df.combo_code.value_counts()

97/15       13436
1569/31      8694
97/3         5514
1761/17      3621
111/3        3374
3464/7       2669
641/9        2216
3226/20      1984
4364/18      1590
3216/20      1401
630/12       1283
2623/4       1179
91/4         1169
3610/2       1136
3251/16      1133
3216/4       1086
157/18       1029
96/3         1027
5742/9        988
5848/4        985
3251/2        983
5743/9        948
4339/7        943
111/1         905
3362/31       891
96/9          888
641/11        873
411/2         831
3058/16       819
5848/9        768
            ...  
1081/9          2
2791/6          2
255/2           2
2659/16         2
4375/4          2
3364/18         2
3583/4          2
6713/13         2
188/16          2
5394/3          2
5394/17         2
32000/0         2
4094/18         1
4334/1          1
3245/2          1
4813/3          1
190/11          1
5745/2          1
3245/4          1
1326/11         1
84/19           1
1322/11         1
4752/7          1
1328/11         1
3241/2    

In [55]:
final_df.shape

(102486, 45)

In [54]:
final_df.isnull().sum()

ESS_ID                            0
eventTimeStamp                    0
eventDescription              15377
actionDescription            102486
ecuSoftwareVersion               37
ecuSerialNumber                   0
ecuModel                          0
ecuMake                           0
ecuSource                         0
SPN                               0
FMI                               0
Active                            0
activeTransitionCount             0
faultValue                   102486
EquipmentID                       0
MCTNumber                         0
Latitude                          0
Longitude                         0
locationTimeStamp                 0
AcceleratorPedal              55990
BarometricPressure            52382
CruiseControlActive           52639
CruiseControlSetSpeed         52103
DistanceLtd                   52289
EngineCoolantTemperature      52358
EngineLoad                    52574
EngineOilPressure             52349
EngineOilTemperature        

In [58]:
final_df = final_df.drop(['actionDescription', 'faultValue', 'ServiceDistance'], axis = 1)

In [72]:
final_df.head()

,ESS_ID,eventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,SPN,FMI,...,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,derated,combo_code
FaultID,,,,,,,,,,,,,,,,,,,,,
9,990442,2015-02-21 11:40:22.000,High (Severity Low) Water In Fuel Indicator,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,97,15,...,True,NaN,1023,NaN,NaN,NaN,NaN,NaN,0,97/15
41,990856,2015-02-21 12:06:22.000,Condition Exists Engine Protection Torque Derate,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,...,True,51.8,18431,False,51.97187,3276.75,0,0.58,1,1569/31
42,990862,2015-02-21 12:06:46.000,High (Severity Low) Water In Fuel Indicator,05290170*03005863*051914190353*09400015*G1*BDR*,79635947,6X1u13D1500000000,CMMNS,0,97,15,...,True,116.6,17407,False,40.65127,3276.75,0,16.53,0,97/15
43,990893,2015-02-21 12:08:43.000,Incorrect Data Engine Intake Manifold #1 Pressure,04993120*00184243*060712145341*07700027*I0*BBZ*,79619432,6X1u10D1500000000,CMMNS,0,102,2,...,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,0,102/2
46,990931,2015-02-21 12:10:51.000,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,...,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,1,5246/0


In [59]:
derated.EquipmentID.value_counts()

1490         408
1505         270
1790         206
1692         169
1480         165
1607         164
105344451    156
1670         156
1636         156
1888         146
1473         128
1645         128
1580         126
1383         126
1395         125
1587         109
1417         107
1711         107
1591         104
1564         100
1577          95
1494          92
1818          91
1573          84
1778          83
1608          82
1966          73
1730          70
1853          70
1787          64
            ... 
1610           2
1514           2
1994           2
1370           2
2017           2
1985           2
1945           2
1625           2
1713           2
1832           2
1914           2
2120           2
2002           2
2027           2
2019           2
1872           2
1995           2
1907           2
1859           2
1699           2
1651           2
1435           2
1944           2
105360462      2
1976           2
307            2
2015           2
105338352     

In [60]:
# joined_df.EquipmentID = joined_df.EquipmentID.astype(int)
worst_truck = joined_df.loc[joined_df.EquipmentID == 1490]

In [61]:
worst_truck.shape

(883, 45)

In [62]:
worst_truck.head()

,ESS_ID,eventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,SPN,...,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,derated,combo_code
FaultID,,,,,,,,,,,,,,,,,,,,,
2866,1024457,2015-02-23 14:43:04.000,NaN,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,4334,...,95,17407,NaN,NaN,66.97216,3276.75,NaN,17.11,0,4334/18
4010,1045409,2015-02-24 11:20:30.000,NaN,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,4334,...,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,0,4334/18
4362,1050117,2015-02-24 14:52:39.000,NaN,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,4334,...,93.2,17407,NaN,NaN,67.07896,3276.75,NaN,15.66,0,4334/18
5490,1067152,2015-02-25 11:27:39.000,NaN,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,4334,...,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,0,4334/18
40490,2204533,2015-05-01 15:18:01.000,NaN,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,4334,...,113,17407,NaN,NaN,60.72932,3276.75,NaN,13.63,0,4334/18


In [63]:
worst_truck.isnull().sum()

ESS_ID                         0
eventTimeStamp                 0
eventDescription             314
actionDescription            883
ecuSoftwareVersion             0
ecuSerialNumber                0
ecuModel                       0
ecuMake                        0
ecuSource                      0
SPN                            0
FMI                            0
Active                         0
activeTransitionCount          0
faultValue                   883
EquipmentID                    0
MCTNumber                      0
Latitude                       0
Longitude                      0
locationTimeStamp              0
AcceleratorPedal             445
BarometricPressure           445
CruiseControlActive          445
CruiseControlSetSpeed        445
DistanceLtd                  445
EngineCoolantTemperature     445
EngineLoad                   445
EngineOilPressure            445
EngineOilTemperature         446
EngineRpm                    445
EngineTimeLtd                445
FuelLevel 

In [64]:
worst_truck.columns

Index(['ESS_ID', 'eventTimeStamp', 'eventDescription', 'actionDescription',
       'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake',
       'ecuSource', 'SPN', 'FMI', 'Active', 'activeTransitionCount',
       'faultValue', 'EquipmentID', 'MCTNumber', 'Latitude', 'Longitude',
       'locationTimeStamp', 'AcceleratorPedal', 'BarometricPressure',
       'CruiseControlActive', 'CruiseControlSetSpeed', 'DistanceLtd',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelRate', 'FuelTemperature', 'IgnStatus',
       'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake',
       'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure', 'derated', 'combo_code'],
      dtype='object')

In [66]:
worst_truck.loc[worst_truck['combo_code'] == '1569/31']

,ESS_ID,eventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,SPN,...,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,derated,combo_code
FaultID,,,,,,,,,,,,,,,,,,,,,
106496,3244863,2015-07-01 07:01:31.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,1569,...,107.6,18431,NaN,NaN,65.07893,3276.75,NaN,11.02,1,1569/31
106618,3246845,2015-07-01 08:27:16.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,1569,...,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN,1,1569/31
114263,3355732,2015-07-08 11:09:22.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,1569,...,123.8,18431,NaN,NaN,61.02059,3276.75,NaN,27.84,1,1569/31
114266,3355773,2015-07-08 11:11:31.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,1569,...,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN,1,1569/31
114347,3357136,2015-07-08 12:11:10.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,1569,...,131,18431,NaN,NaN,53.82628,3276.75,NaN,26.1,1,1569/31
114356,3357274,2015-07-08 12:17:06.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,1569,...,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN,1,1569/31
114417,3358524,2015-07-08 13:09:55.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,1569,...,116.6,18431,NaN,NaN,68.27316,3276.75,NaN,31.03,1,1569/31
114420,3358569,2015-07-08 13:12:05.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,1569,...,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN,1,1569/31
114494,3359526,2015-07-08 13:41:58.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,1569,...,141.8,18431,NaN,NaN,33.56376,3276.75,NaN,27.84,1,1569/31


In [68]:
worst_truck2 = joined_df.loc[joined_df.EquipmentID == 1505]
worst_truck2.loc[worst_truck2['combo_code'] == '1569/31']

,ESS_ID,eventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,SPN,...,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,derated,combo_code
FaultID,,,,,,,,,,,,,,,,,,,,,
593220,11782624,2016-09-23 17:02:51.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00098289*082113134117*07700053*I0*BBZ*,79519586,6X1u10D1500000000,CMMNS,0,1569,...,98.6,18431,NaN,NaN,64.53522,NaN,NaN,NaN,1,1569/31
593244,11783165,2016-09-23 17:34:50.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00098289*082113134117*07700053*I0*BBZ*,79519586,6X1u10D1500000000,CMMNS,0,1569,...,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN,1,1569/31
593252,11783324,2016-09-23 17:38:13.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00098289*082113134117*07700053*I0*BBZ*,79519586,6X1u10D1500000000,CMMNS,0,1569,...,102.2,18431,NaN,NaN,64.95271,NaN,NaN,NaN,1,1569/31
593324,11785611,2016-09-23 21:13:01.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00098289*082113134117*07700053*I0*BBZ*,79519586,6X1u10D1500000000,CMMNS,0,1569,...,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN,1,1569/31
593327,11785740,2016-09-23 21:29:23.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00098289*082113134117*07700053*I0*BBZ*,79519586,6X1u10D1500000000,CMMNS,0,1569,...,95,18431,NaN,NaN,17.86442,NaN,NaN,NaN,1,1569/31
593354,11786822,2016-09-24 01:12:55.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00098289*082113134117*07700053*I0*BBZ*,79519586,6X1u10D1500000000,CMMNS,0,1569,...,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN,1,1569/31
593518,11789836,2016-09-24 07:39:04.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00098289*082113134117*07700053*I0*BBZ*,79519586,6X1u10D1500000000,CMMNS,0,1569,...,96.8,18431,NaN,NaN,50.83593,NaN,NaN,NaN,1,1569/31
593577,11791190,2016-09-24 09:12:56.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00098289*082113134117*07700053*I0*BBZ*,79519586,6X1u10D1500000000,CMMNS,0,1569,...,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN,1,1569/31
593585,11791373,2016-09-24 09:14:08.000,Condition Exists Engine Protection Torque Derate,NaN,04993120*00098289*082113134117*07700053*I0*BBZ*,79519586,6X1u10D1500000000,CMMNS,0,1569,...,111.2,18431,NaN,NaN,0,NaN,NaN,NaN,1,1569/31


### Seems to be a pattern with the worst trucks getting repeated derate codes

In [76]:
testing_df = pd.get_dummies(final_df['combo_code'])
testing_df['derated'] = final_df['derated']
testing_df['eventTimeStamp'] = final_df['eventTimeStamp']
testing_df['EquipmentID'] = final_df['EquipmentID']
testing_df.head()

,100/1,100/18,100/2,100/3,100/4,101/0,101/15,101/16,101/2,101/3,...,95/16,96/3,96/9,97/15,97/16,97/3,97/4,derated,eventTimeStamp,EquipmentID
FaultID,,,,,,,,,,,,,,,,,,,,,
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,2015-02-21 11:40:22.000,1429
41,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2015-02-21 12:06:22.000,1721
42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,2015-02-21 12:06:46.000,1610
43,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2015-02-21 12:08:43.000,1576
46,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2015-02-21 12:10:51.000,1395
